# Analise de influencia das variaveis no churn dos clientes

**Objetivo:** Identificar quais variáveis estão diretamente relacionadas com o churn de clientes, através de métodos como  analise de correlação, regressões múltiplas e análise de sensibilidade. 


## 0. Imports, carga e preparação de dados

In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
from matplotlib import pyplot as plt
import seaborn as sns

## 1. Correlações

As correlações podem ser extraídas diretamente de ferramentas de auto-report, que além de correlações lineares e não lineares nos fornecem uma boa visão geral dos dados.

In [46]:
df = pd.read_csv('../Data/churn.csv')
#profile = ProfileReport(df = df, title = 'churn_eda_report')
categorical_columns = ['gender','Partner','Dependents','PhoneService','MultipleLines','InternetService',
                       'OnlineSecurity','OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 
                       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod','Churn']

In [47]:
df.sample(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
5887,4316-XCSLJ,Male,0,No,Yes,17,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,50.3,846.8,No
2793,3904-UKFRE,Male,0,No,No,4,Yes,Yes,Fiber optic,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,80.1,336.15,No
2400,5141-ZUVBH,Female,0,No,Yes,9,Yes,No,Fiber optic,Yes,...,Yes,Yes,No,Yes,Month-to-month,No,Bank transfer (automatic),93.0,870.25,No


In [67]:
df.OnlineBackup.value_counts()

OnlineBackup
No                     3088
Yes                    2429
No internet service    1526
Name: count, dtype: int64

In [69]:
df_categorical = df.copy()
dict_categorical_reference = {}
special_cases = ['No phone service', 'No internet service']


def default_conversion(df_categorical, column, values):
    for value in values:
        if value not in special_cases:
            df_categorical[column + '_' + value] = df[column].apply(lambda x: 1 if x == value else 0)
        else:
            df_categorical[value] = df[column].apply(lambda x: 1 if x == 'Yes' else 0)
            

for c in categorical_columns:
    dict_categorical_to_numeric = {}
    values = df[c].unique()
    
    if len(values) > 2:

        default_conversion(df_categorical = df_categorical, values = values, column = c)                
        df_categorical.drop(columns = [c],inplace=True)
    
    else:
        for idx, value in enumerate(values):
            dict_categorical_to_numeric[value] = idx
        
        dict_categorical_reference[c] = dict_categorical_to_numeric
        df_categorical[c] = df[c].apply(lambda x: dict_categorical_to_numeric[x])
    

In [77]:
a = set(['a','b','yu'])
b = set(['a','b','c','d','e','g','f'])
len(a)

3

In [70]:
df_categorical.sample(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,...,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic)
2560,8178-EYZUO,1,0,1,0,46,1,0,93.7,4154.8,...,0,0,1,1,0,0,1,0,0,0
1597,5327-CNLUQ,1,0,0,0,48,1,0,96.9,4473.45,...,1,1,0,0,0,1,0,0,1,0
1355,5624-RYAMH,0,0,1,0,9,1,0,19.5,178.85,...,0,0,0,1,0,0,1,0,0,0


In [71]:
df_categorical.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'PaperlessBilling', 'MonthlyCharges',
       'TotalCharges', 'Churn', 'No phone service', 'MultipleLines_No',
       'MultipleLines_Yes', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineSecurity_No', 'OnlineSecurity_Yes', 'No internet service',
       'OnlineBackup_Yes', 'OnlineBackup_No', 'DeviceProtection_No',
       'DeviceProtection_Yes', 'TechSupport_No', 'TechSupport_Yes',
       'StreamingTV_No', 'StreamingTV_Yes', 'StreamingMovies_No',
       'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaymentMethod_Electronic check',
       'PaymentMethod_Mailed check', 'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit card (automatic)'],
      dtype='object')

Verificando os campos numericos em numericos (até então estão tipo object)

In [81]:
for item in df.columns.values:
    if item not in categorical_columns:
        print('='*30)
        try:
            df_categorical[item] = df_categorical[item].apply(lambda x: float(x))
            print('{} --> Ok'.format(item))
        except Exception as e:
            print('{} -- FALHOU'.format(item))
            print(e)
        print('='*30)

customerID -- FALHOU
could not convert string to float: '7590-VHVEG'
SeniorCitizen --> Ok
tenure --> Ok
MonthlyCharges --> Ok
TotalCharges -- FALHOU
could not convert string to float: ' '


In [82]:
df_categorical[df_categorical.TotalCharges == ' '].tenure.value_counts()

tenure
0.0    11
Name: count, dtype: int64

Quando não há vencimentos o totalcharges é nulo

In [83]:
# Adaptando conversão de TotalCharges
df_categorical['TotalCharges'] = df['TotalCharges'].apply(lambda x: float(x) if x != ' ' else 0)


In [55]:
df_categorical.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'PaperlessBilling', 'MonthlyCharges',
       'TotalCharges', 'Churn', 'MultipleLines_No phone service',
       'MultipleLines_No', 'MultipleLines_Yes', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineSecurity_No', 'OnlineSecurity_Yes',
       'OnlineSecurity_No internet service', 'OnlineBackup_Yes',
       'OnlineBackup_No', 'OnlineBackup_No internet service',
       'DeviceProtection_No', 'DeviceProtection_Yes',
       'DeviceProtection_No internet service', 'TechSupport_No',
       'TechSupport_Yes', 'TechSupport_No internet service', 'StreamingTV_No',
       'StreamingTV_Yes', 'StreamingTV_No internet service',
       'StreamingMovies_No', 'StreamingMovies_Yes',
       'StreamingMovies_No internet service', 'Contract_Month-to-month',
       'Contract_One year', 'Contract_Two year',
       'PaymentMethod_Electronic check', 'Payment

### 1.1 Linear

In [53]:
pear_corr = df_categorical.iloc[:,1:].corr(method='pearson')
pear_corr

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,...,StreamingMovies_No,StreamingMovies_Yes,StreamingMovies_No internet service,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic)
gender,1.000000,-0.001874,0.001808,0.010517,0.005106,-0.006488,0.011754,-0.014569,-0.000080,-0.008612,...,0.005374,-0.010487,0.006026,-0.003386,0.008026,-0.003695,0.000752,0.013744,-0.016024,0.001215
SeniorCitizen,-0.001874,1.000000,-0.016479,-0.211185,0.016567,0.008576,-0.156530,0.220173,0.103006,0.150889,...,0.034210,0.120176,-0.182742,0.138360,-0.046262,-0.117000,0.171718,-0.153477,-0.016159,-0.024135
Partner,0.001808,-0.016479,1.000000,-0.452676,-0.379697,-0.017706,-0.014877,-0.096848,-0.317504,0.150448,...,0.117529,-0.117412,-0.000615,0.280865,-0.082783,-0.248091,0.083852,0.095125,-0.110706,-0.082029
Dependents,0.010517,-0.211185,-0.452676,1.000000,0.159712,-0.001762,0.111377,-0.113890,0.062078,-0.164221,...,-0.078198,-0.039741,0.139812,-0.231720,0.068368,0.204613,-0.150642,0.059071,0.052021,0.060267
tenure,0.005106,0.016567,-0.379697,0.159712,1.000000,0.008448,-0.006152,0.247900,0.826178,-0.352229,...,-0.252220,0.286111,-0.039062,-0.645561,0.202570,0.558533,-0.208363,-0.233852,0.243510,0.233006
PhoneService,-0.006488,0.008576,-0.017706,-0.001762,0.008448,1.000000,-0.016505,0.247398,0.113214,0.011942,...,-0.112254,-0.032959,0.172209,-0.000742,-0.002791,0.003519,0.003062,-0.003319,0.007556,-0.007721
PaperlessBilling,0.011754,-0.156530,-0.014877,0.111377,-0.006152,-0.016505,1.000000,-0.352150,-0.158574,-0.191825,...,-0.059488,-0.211716,0.321013,-0.169096,0.051391,0.147889,-0.208865,0.205398,0.016332,0.013589
MonthlyCharges,-0.014569,0.220173,-0.096848,-0.113890,0.247900,0.247398,-0.352150,1.000000,0.651174,0.193356,...,0.018075,0.627429,-0.763557,0.060165,0.004904,-0.074681,0.271625,-0.377437,0.042812,0.030550
TotalCharges,-0.000080,0.103006,-0.317504,0.062078,0.826178,0.113214,-0.158574,0.651174,1.000000,-0.198324,...,-0.202188,0.520122,-0.375223,-0.444255,0.170814,0.354481,-0.059246,-0.295758,0.185987,0.182915
Churn,-0.008612,0.150889,0.150448,-0.164221,-0.352229,0.011942,-0.191825,0.193356,-0.198324,1.000000,...,0.130845,0.061382,-0.227890,0.405103,-0.177820,-0.302253,0.301919,-0.091683,-0.117937,-0.134302


### 1.2 Não Linear

In [84]:
spear_corr = df_categorical.iloc[:,1:].corr(method='spearman')
separ_corr_churn = spear_corr['Churn']
plt.figure(figsize=(16,6))
sns.heatmap(separ_corr_churn ,annot=True)


IndexError: Inconsistent shape between the condition and the input (got (36, 1) and (36,))

<Figure size 1600x600 with 0 Axes>

In [87]:
separ_corr_churn[separ_corr_churn >=0.15].sort_values()

Partner                           0.150448
SeniorCitizen                     0.150889
MonthlyCharges                    0.184743
DeviceProtection_No               0.252481
OnlineBackup_No                   0.268005
PaymentMethod_Electronic check    0.301919
InternetService_Fiber optic       0.308020
TechSupport_No                    0.337281
OnlineSecurity_No                 0.342637
Contract_Month-to-month           0.405103
Churn                             1.000000
Name: Churn, dtype: float64

In [16]:
df['InternetService'].value_counts()

InternetService
Fiber optic    3096
DSL            2421
No             1526
Name: count, dtype: int64

In [18]:
dict_internet = {'Fiber optic' :  0,
'DSL' :  1,
'No' :  2 }

df_categorical.InternetService = df['InternetService'].apply(lambda x: dict_internet[x])

## 2. Regressões múltiplas

## 3. Análise de sensibilidade